In [1]:
from typing import List, Optional, Any
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

# Configuration for Ollama using OpenAI-compatible endpoint
OLLAMA_BASE_URL = "http://localhost:11434"
MODEL_NAME = "qwen3:8b"  # Updated to use available model

# Ollama provides OpenAI-compatible API at /v1/ endpoint
ollama_model = OpenAIModel(
    model_name=MODEL_NAME, provider=OpenAIProvider(base_url=f"{OLLAMA_BASE_URL}/v1")
)

print("✅ Imports completed successfully!")
print(f"🌐 Ollama endpoint: {OLLAMA_BASE_URL}")
print(f"🤖 Model: {MODEL_NAME}")
print("💡 Using OpenAI-compatible interface with Ollama")

✅ Imports completed successfully!
🌐 Ollama endpoint: http://localhost:11434
🤖 Model: qwen3:8b
💡 Using OpenAI-compatible interface with Ollama


In [2]:
class CityLocation(BaseModel):
    city: str
    country: str


ollama_model = OpenAIModel(
    model_name=MODEL_NAME, provider=OpenAIProvider(base_url=f"{OLLAMA_BASE_URL}/v1")
)
agent = Agent(ollama_model, output_type=CityLocation)

result = await agent.run('Where were the olympics held in 2012?')
print(result.output)
#> city='London' country='United Kingdom'
print(result.usage())
#> Usage(requests=1, request_tokens=57, response_tokens=8, total_tokens=65)

city='London' country='United Kingdom'
Usage(requests=1, request_tokens=163, response_tokens=157, total_tokens=320)


In [3]:
# Let's first check what models are available in Ollama
import requests
import json

try:
    response = requests.get(f"{OLLAMA_BASE_URL}/api/tags")
    if response.status_code == 200:
        models = response.json()
        print("Available models in Ollama:")
        for model in models.get('models', []):
            print(f"  - {model['name']}")
    else:
        print(f"Failed to get models: {response.status_code}")
except Exception as e:
    print(f"Error connecting to Ollama: {e}")
    print("Make sure Ollama is running on localhost:11434")

Available models in Ollama:
  - qwen3:8b
  - gemma3:4b-it-qat
  - deepseek-r1:8b
  - hf.co/unsloth/DeepSeek-R1-0528-Qwen3-8B-GGUF:Q4_K_XL
  - deepseek-r1-qwen3-8b:latest
  - llama3.2:1b


In [4]:
# Let's switch to a model that supports function calling
# llama3.2:1b should support tools better than deepseek-r1
# MODEL_NAME = "llama3.2:1b"
MODEL_NAME = "qwen3:8b"
print(f"🔄 Switching to model: {MODEL_NAME}")
print("📝 This model should support function calling/tools")

🔄 Switching to model: qwen3:8b
📝 This model should support function calling/tools


In [7]:
from pydantic_ai.common_tools.duckduckgo import duckduckgo_search_tool

# Create agent with the new model
ollama_model = OpenAIModel(
    model_name=MODEL_NAME, 
    provider=OpenAIProvider(base_url=f"{OLLAMA_BASE_URL}/v1")
)
agent = Agent(
    ollama_model,
    tools=[duckduckgo_search_tool()],
    system_prompt='Search DuckDuckGo for the given query and return the results.',
)

result = await agent.run(
    'Can you list the top five highest-grossing animated films of 2025?'
)
print(result.output)
print(result.usage())

<think>
Okay, so the user asked for the top five highest-grossing animated films of 2025. Let me check the search results to see what's available.

Looking at the first result from whatsafterthemovie.com, it lists Ne Zha as number 1 with a domestic gross of $20,858,156. But wait, that's just domestic, and the worldwide might be higher. The second result from Wikipedia mentions a list of the 10 highest-grossing animated films of 2025, but the exact top five aren't detailed there. 

The third result from simplebeen.com talks about Disney's dominance and lists the top 10, but again, the specific top five aren't clear. The fourth result from listchallenges.com mentions Ne Zha 2 as number 1 and lists the top 50, but the top five aren't fully provided. 

The fifth result is an IMDb list, but it's incomplete, only showing some entries. The sixth result from justjared.com states that Ne Zha 2 is the highest-grossing with $1.72 billion. The seventh result from animesoulking.com also mentions Ne

In [8]:
# Example 1: Adding tool functions to a PydanticAI Agent
from datetime import datetime
import math
from pydantic_ai import RunContext

# Define response models
class WeatherInfo(BaseModel):
    location: str
    temperature: float
    description: str
    timestamp: str

class MathResult(BaseModel):
    operation: str
    result: float
    explanation: str

# Create agent with the new model
ollama_model = OpenAIModel(
    model_name=MODEL_NAME, 
    provider=OpenAIProvider(base_url=f"{OLLAMA_BASE_URL}/v1")
)

# Create agent - we'll add tools to this agent
agent = Agent(ollama_model)

# Method 1: Adding tools using @agent.tool decorator
@agent.tool
def get_weather(ctx: RunContext, location: str) -> str:
    """Get current weather for a location."""
    # This is a mock function - in real use, you'd call a weather API
    mock_weather = {
        "London": {"temp": 15, "desc": "Cloudy"},
        "Paris": {"temp": 18, "desc": "Sunny"},
        "New York": {"temp": 12, "desc": "Rainy"},
        "Tokyo": {"temp": 22, "desc": "Clear"}
    }
    
    weather = mock_weather.get(location, {"temp": 20, "desc": "Unknown"})
    return f"The weather in {location} is {weather['temp']}°C and {weather['desc']}"

@agent.tool
def calculate_circle_area(ctx: RunContext, radius: float) -> str:
    """Calculate the area of a circle given its radius."""
    if radius <= 0:
        return "Radius must be positive"
    
    area = math.pi * radius ** 2
    return f"The area of a circle with radius {radius} is {area:.2f} square units"

@agent.tool
def get_current_time(ctx: RunContext) -> str:
    """Get the current date and time."""
    now = datetime.now()
    return f"Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

print("✅ Agent with tools created successfully!")
print("🔧 Available tools:")
print("  - get_weather(location)")
print("  - calculate_circle_area(radius)")
print("  - get_current_time()")
print("\n🎯 The agent can now use these tools to answer questions!")

✅ Agent with tools created successfully!
🔧 Available tools:
  - get_weather(location)
  - calculate_circle_area(radius)
  - get_current_time()

🎯 The agent can now use these tools to answer questions!


In [9]:
# Test the agent with tools
print("🧪 Testing the agent with different queries that should trigger tools:\n")

# Test 1: Weather query
print("1️⃣ Testing weather tool:")
result1 = await agent.run("What's the weather like in London?")
print(f"Response: {result1.data}")
print(f"Usage: {result1.usage()}")
print()

🧪 Testing the agent with different queries that should trigger tools:

1️⃣ Testing weather tool:
Response: <think>
Okay, let me process the user's query and the tool response. The user asked, "What's the weather like in London?" I called the get_weather function with London as the location. The response came back as 15°C and Cloudy. Now I need to present this information clearly.

First, I'll confirm the location to make sure there's no confusion. Then, state the temperature and the weather condition. I should keep it straightforward and friendly. Maybe add a sentence about the weather being mild or comfortable based on the temperature. Let me check if there's any additional info needed, but since the user only asked for the current weather, sticking to the provided details is best. Alright, time to put it all together in a natural sentence.
</think>

The current weather in London is 15°C with cloudy conditions. It looks like a mild and comfortable day!
Usage: Usage(requests=2, request

/tmp/ipykernel_1269940/3906904202.py:7: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  print(f"Response: {result1.data}")


In [10]:
# Test 2: Math calculation
print("2️⃣ Testing math tool:")
result2 = await agent.run("What's the area of a circle with radius 5?")
print(f"Response: {result2.output}")
print()

# Test 3: Current time
print("3️⃣ Testing time tool:")
result3 = await agent.run("What time is it now?")
print(f"Response: {result3.output}")
print()

2️⃣ Testing math tool:
Response: <think>
Okay, let me check the user's question again. They asked for the area of a circle with radius 5. The tool response says the area is 78.54 square units. Wait, the function calculate_circle_area was called with radius 5, and the result is 78.54. That makes sense because π times 5 squared is approximately 3.1416 * 25 = 78.54. So the answer is correct. I should present this in a clear way to the user, maybe mention the formula used and the calculation steps. Let me make sure to format the response properly and confirm the answer.
</think>

The area of a circle with a radius of **5.0** is **78.54 square units**.

This is calculated using the formula:  
$$
\text{Area} = \pi \times r^2 = \pi \times 5^2 \approx 3.1416 \times 25 \approx 78.54
$$

Let me know if you need further clarification! 🌟

3️⃣ Testing time tool:
Response: <think>
</think>

The current time is 2025-06-17 at 14:32:22.



In [11]:
# 🎯 COMPREHENSIVE GUIDE: Different ways to add tool functions in PydanticAI

print("="*60)
print("📚 TOOL FUNCTIONS GUIDE FOR PYDANTIC AI")
print("="*60)

# Method 1: Using @agent.tool decorator (already shown above)
print("\n✅ Method 1: @agent.tool decorator")
print("   ✓ Directly decorate functions")
print("   ✓ Functions must take RunContext as first parameter")
print("   ✓ Best for simple tools")

# Method 2: Creating standalone tools and adding them to agent
from pydantic_ai.tools import Tool

def database_query(ctx: RunContext, table: str, condition: str) -> str:
    """Mock database query function."""
    # This would normally connect to a real database
    mock_results = {
        "users": ["Alice", "Bob", "Charlie"],
        "products": ["Laptop", "Phone", "Tablet"],
        "orders": ["Order #1", "Order #2", "Order #3"]
    }
    results = mock_results.get(table, ["No data found"])
    return f"Query results from {table} where {condition}: {', '.join(results)}"

# Create a standalone tool
db_tool = Tool(database_query)

# Create new agent and add the tool
agent2 = Agent(ollama_model)
agent2._register_tool(db_tool)

print("\n✅ Method 2: Standalone Tool objects")
print("   ✓ Create Tool objects explicitly")
print("   ✓ More control over tool configuration")
print("   ✓ Can be reused across multiple agents")

# Method 3: Tools with complex return types
@agent2.tool
def search_files(ctx: RunContext, pattern: str, file_type: str = "txt") -> dict:
    """Search for files matching a pattern."""
    mock_files = {
        "txt": ["document1.txt", "notes.txt", "readme.txt"],
        "py": ["main.py", "utils.py", "test.py"],
        "md": ["README.md", "CHANGELOG.md"]
    }
    
    files = mock_files.get(file_type, [])
    matching = [f for f in files if pattern.lower() in f.lower()]
    
    return {
        "pattern": pattern,
        "file_type": file_type,
        "matches": matching,
        "total_found": len(matching)
    }

print("\n✅ Method 3: Tools with complex return types")
print("   ✓ Return dictionaries, lists, or custom objects")
print("   ✓ Agent will serialize appropriately")

# Method 4: Tools with error handling
@agent2.tool
def safe_division(ctx: RunContext, dividend: float, divisor: float) -> str:
    """Safely divide two numbers with error handling."""
    try:
        if divisor == 0:
            return "Error: Cannot divide by zero"
        result = dividend / divisor
        return f"{dividend} ÷ {divisor} = {result:.4f}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

print("\n✅ Method 4: Tools with error handling")
print("   ✓ Handle errors gracefully")
print("   ✓ Return meaningful error messages")
print("   ✓ Prevent agent crashes")

print("\n🎉 All tool methods demonstrated!")
print("💡 Key Points:")
print("   • Always include RunContext as first parameter")
print("   • Use descriptive docstrings - the agent uses them")
print("   • Handle errors gracefully in your tools")
print("   • Tools can return strings, dicts, or complex objects")
print("   • Test your tools individually before using with agent")

📚 TOOL FUNCTIONS GUIDE FOR PYDANTIC AI

✅ Method 1: @agent.tool decorator
   ✓ Directly decorate functions
   ✓ Functions must take RunContext as first parameter
   ✓ Best for simple tools

✅ Method 2: Standalone Tool objects
   ✓ Create Tool objects explicitly
   ✓ More control over tool configuration
   ✓ Can be reused across multiple agents

✅ Method 3: Tools with complex return types
   ✓ Return dictionaries, lists, or custom objects
   ✓ Agent will serialize appropriately

✅ Method 4: Tools with error handling
   ✓ Handle errors gracefully
   ✓ Return meaningful error messages
   ✓ Prevent agent crashes

🎉 All tool methods demonstrated!
💡 Key Points:
   • Always include RunContext as first parameter
   • Use descriptive docstrings - the agent uses them
   • Handle errors gracefully in your tools
   • Tools can return strings, dicts, or complex objects
   • Test your tools individually before using with agent


In [12]:
# Test the new tools
print("🧪 Testing additional tools:")

# Test database tool
print("\n📊 Testing database query tool:")
result_db = await agent2.run("Search for users in the database where active = true")
print(f"Response: {result_db.output}")

# Test file search tool
print("\n🔍 Testing file search tool:")
result_files = await agent2.run("Find all Python files containing 'main'")
print(f"Response: {result_files.output}")

# Test division tool
print("\n➗ Testing safe division tool:")
result_div = await agent2.run("What is 100 divided by 7?")
print(f"Response: {result_div.output}")

print("\n✨ All tools working correctly!")

🧪 Testing additional tools:

📊 Testing database query tool:
Response: <think>
Okay, let me see. The user asked to search for users in the database where active is true. I called the database_query function with table 'users' and condition 'active = true'. The response came back with the results: Alice, Bob, Charlie. Now I need to present these results to the user in a clear way.

First, I should confirm that the query was executed successfully. Then list out the active users. Maybe use a friendly message to inform them of the results. Let me check if there are any other steps needed, but since the user just wanted the search, providing the list should be sufficient. I'll format the response to be easy to read.
</think>

Here are the active users found in the database:

- Alice
- Bob
- Charlie

Let me know if you need any additional information!

🔍 Testing file search tool:
Response: <think>
Okay, the user asked to find all Python files containing 'main'. I used the search_files functio